## Suivi des métriques avec MLflow

Lorsque vous entraînez un modèle avec un script, vous pouvez inclure MLflow dans vos scripts pour suivre tous les paramètres, métriques et artefacts. Lorsque vous exécutez le script en tant que travail dans Azure Machine Learning, vous pouvez passer en revue tous les paramètres d’entrée et les sorties pour chaque exécution.

### Comprendre MLflow
MLflow est une plateforme open source conçue pour gérer le cycle de vie complet du Machine Learning. Comme elle est open source, elle peut être utilisée lors de la formation de modèles sur différentes plateformes. Ici, nous allons nous pencher sur la façon dont nous pouvons intégrer MLflow à des travaux Azure Machine Learning.

Il existe deux options pour suivre les travaux de Machine Learning avec MLflow :

Activer la journalisation active à l’aide de mlflow.autolog()
Utilisez les fonctions de journalisation pour suivre des métriques personnalisées à l’aide de mlflow.log_*
Avant de pouvoir utiliser l’une de ces options, vous devez configurer l’environnement pour utiliser MLflow.

### Inclure MLflow dans l’environnement
Pour utiliser MLflow pendant le travail de formation, les packages pip mlflow et azureml-mlflow doivent être installés sur le calcul qui exécute le script. Par conséquent, vous devez inclure ces deux packages dans l’environnement. Vous pouvez créer un environnement en faisant référence à un fichier YAML qui décrit l’environnement Conda. Dans le cadre de l’environnement Conda, vous pouvez inclure ces deux packages.

Par exemple, dans cet environnement personnalisé, mlflow et azureml-mlflow sont installés à l’aide de pip :
```yaml
name: mlflow-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip
  - pip:
    - numpy
    - pandas
    - scikit-learn
    - matplotlib
    - mlflow
    - azureml-mlflow
```

### Activer la journalisation automatique
Lorsque vous utilisez l’une des bibliothèques courantes pour le Machine Learning, vous pouvez activer la journalisation de la connexion dans MLflow. La journalisation automatique enregistre les paramètres, les métriques et les artefacts de modèle sans qu’il soit nécessaire de spécifier ce qui doit être consigné.

La journalisation automatique est prise en charge pour les bibliothèques suivantes :

- Scikit-learn
- TensorFlow et Keras
- XGBoost
- LightGBM
- Spark
- Fastai
- Pytorch
Pour activer la journalisation automatique, ajoutez le code suivant à votre script de formation :
```python
import mlflow
mlflow.autolog()
```

### Journaliser des métriques avec MLflow
Dans votre script de formation, vous pouvez décider des métriques personnalisées à consigner avec MLflow.

Selon le type de valeur que vous souhaitez consigner, utilisez la commande MLflow pour stocker la métrique avec l’exécution d’expérimentation :
- `mlflow.log_param()` : Consignez un paramètre de valeur de clé unique. Utilisez cette fonction pour un paramètre d’entrée que vous souhaitez consigner.
- `mlflow.log_metric()`: Consignez une métrique de valeur de clé unique. La valeur doit être un nombre. Utilisez cette fonction pour toute sortie que vous souhaitez stocker avec l’exécution.
- `mlflow.log_artifact()` : Consigner un fichier. Utilisez cette fonction pour tout tracé que vous souhaitez consigner et enregistrer en tant que fichier image avant.
Pour ajouter MLflow à un script de formation existant, vous pouvez ajouter le code suivant :
```python
import mlflow

mlflow.log_param("param1", value1)
mlflow.log_metric("rmse", rmse)
mlflow.log_artifact("plot.png")
```

```python
import mlflow

reg_rate = 0.1
mlflow.log_param("Regularization rate", reg_rate)
```

## Récupérer les métriques avec MLflow dans un notebook
Lorsque vous exécutez un script d’entraînement en tant que travail dans Azure Machine Learning et que vous suivez l’entraînement de votre modèle avec MLflow, vous pouvez interroger les exécutions dans un notebook à l’aide de MLflow. L’utilisation de MLflow dans un notebook vous permet de mieux contrôler les exécutions que vous souhaitez récupérer pour les comparer.

Lorsque vous utilisez MLflow pour interroger vos exécutions, vous faites référence aux expériences et aux exécutions.

### Répertorier toutes les expériences
Vous pouvez obtenir toutes les expériences actives dans l’espace de travail à l’aide de MLflow :
```python
import mlflow

# Get a list of all runs in the experiment
experiments = mlflow.list_experiments()
for exp in experiments:
    print(exp.name)

# Get
experiments = mlflow.list_experiments(max_results=2)
for exp in experiments:
    print(exp.name)



#Si vous souhaitez également récupérer les expériences archivées, incluez l’option ViewType.ALL :
from mlflow.entities import ViewType

experiments = mlflow.list_experiments(view_type=ViewType.ALL)
for exp in experiments:
    print(exp.name)

# Pour récupérer une expérience spécifique, vous pouvez exécuter :
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)
```

### Récupérer des exécutions
MLflow vous permet de rechercher des exécutions au sein d’une expérience. Vous avez besoin de l’ID de l’expérience ou de son nom.

Par exemple, lorsque vous souhaitez récupérer les métriques d’une exécution spécifique :
```python
import mlflow
mlflow.search_runs(exp.experiment_id)

# Get the run id for the experiment and model name
run_id = mlflow.search_runs(exp.experiment_id, filter_string="tags.mlflow.runName = 'model'").iloc[0].run_id

# Get the metrics for the run
mlflow.get_run(run_id).data.metrics
```

Vous pouvez effectuer des recherches sur plusieurs expériences si nécessaire. La recherche à travers les expériences peut être utile si vous souhaitez comparer les exécutions d’un même modèle lorsqu’il est journalisé dans différentes expériences (par différentes personnes ou différentes itérations de projet).

Vous pouvez utiliser search_all_experiments=True si vous souhaitez effectuer une recherche sur toutes les expériences de l’espace de travail.

Par défaut, les expériences sont triées par ordre décroissant de start_time, c’est-à-dire le moment où l’expérience a été mise en file d’attente dans Azure Machine Learning. Toutefois, vous pouvez modifier cette valeur par défaut à l’aide du paramètre order_by.

Par exemple, si vous souhaitez trier par heure de début et afficher uniquement les deux derniers résultats :
```python   
import mlflow
#mlflow.search_runs(order_by=["metrics.accuracy DESC"], max_results=2)
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)
```


Vous pouvez également rechercher une exécution avec une combinaison spécifique dans les hyperparamètres :
```python
mlflow.search_runs(
    exp.experiment_id, filter_string="params.num_boost_round='100'", max_results=2
)
```